## Siamese Neural Networks 
### for Supervised Clustering of High Dimensional Spaces

In [1]:
!python -V
!which python

Python 3.6.10
/Users/seanmacrae/mobius/.venv/bin/python


In [2]:
# from palmerpenguins import load_penguins
# penguins = load_penguins()

# print(penguins.shape)
# penguins.head()

In [3]:
import pandas as pd

# DOWNLOAD DATSET HERE: https://www.kaggle.com/husainsb/lendingclub-issued-loans
loans = pd.read_csv("../data/lc_loan.csv")

print(loans.shape)
loans.head()

/Users/seanmacrae/mobius/.venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (19,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(887379, 74)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# loans["default"] = loans["loan_status"] == "Default"
loans["issue_year"] = [int(x[1]) for x in loans["issue_d"].str.split("-")]

loans = loans[loans["issue_year"] == 2015]
loans.dropna(axis=1, how="all", inplace=True)

In [5]:
loans.shape

(421094, 75)

In [6]:
# sns.pairplot(loans, hue='species')

In [7]:
from sklearn.model_selection import train_test_split

loans_sample, _ = train_test_split(
    loans,
    test_size=0.97,
    stratify=loans["loan_status"])

In [8]:
loans_sample.shape

(12632, 75)

## Tabular Learner

Before we train the Tabular Siamese Learner we will train baseline Tabular Learner for species classification... (why do we do this, exactly? can we just instantiate a Tabular Siamese Learner without a baseline Tabular Learner ???)

Ah yes, to init a new `TabularSiameseModel` we need to provide an `encoder` and `head` and the Tabular Learner will act as the `encoder` we init the `TabularSiameseModel` with.

In [9]:
# # from sklearn.model_selection import train_test_split

# penguins_train, penguins_test = train_test_split(
#     penguins,
#     test_size=0.10,
#     stratify=penguins["species"])

# penguins_train.shape, penguins_test.shape

# df = penguins_train.copy()

# exclude_vars = ["year"]

In [10]:
from sklearn.model_selection import train_test_split

loans_train, loans_test = train_test_split(
    loans_sample,
    test_size=0.10,
    stratify=loans_sample["loan_status"])

loans_train.shape, loans_test.shape

df = loans_train.copy()

exclude_vars = ["id", "member_id", "loan_status", "url", "last_pymnt_d", "next_pymnt_d", "policy_code", "issue_year"]

In [11]:
from fastai.tabular.all import CategoryBlock
                                
# y_names = ["species"]
y_names = ["loan_status"]
y_block = CategoryBlock()

In [12]:
from mobius.utils import emb_sz_rule

cat_names = [x for x in df.select_dtypes(exclude=['int', 'float']).columns if x != y_names]
cat_names = [x for x in cat_names if x not in exclude_vars]

# calc embedding sizes for each categorical feature
emb_szs = {k: emb_sz_rule(len(df[k].unique())) for k in cat_names}
emb_szs

{'term': 2,
 'grade': 5,
 'sub_grade': 12,
 'emp_title': 212,
 'emp_length': 6,
 'home_ownership': 3,
 'verification_status': 3,
 'issue_d': 6,
 'pymnt_plan': 2,
 'desc': 3,
 'purpose': 7,
 'title': 7,
 'zip_code': 68,
 'addr_state': 14,
 'earliest_cr_line': 53,
 'initial_list_status': 2,
 'last_credit_pull_d': 7,
 'application_type': 2,
 'verification_status_joint': 3}

In [13]:
import numpy as np

cont_names = [x for x in df.select_dtypes([np.number]).columns if x != y_names]
cont_names = [x for x in cont_names if x not in exclude_vars]
cont_names

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'annual_inc_joint',
 'dti_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m']

In [14]:
from fastai.tabular.all import (Categorify, CategoryBlock, FillMissing,
                                Normalize, TabDataLoader, TabularPandas,
                                tabular_config, tabular_learner)

procs = [FillMissing, Categorify, Normalize]

In [15]:
from fastai.data.core import range_of
from fastai.tabular.all import RandomSplitter

# # train/test split
splits = RandomSplitter(valid_pct=0.10)(range_of(df))

In [16]:
tabular_pandas = TabularPandas(
        df,
        procs=procs,
        cat_names=cat_names,
        cont_names=cont_names,
        y_names=y_names,
        y_block=y_block,
        splits=splits,
        device="cpu")

In [17]:
trn_dl = TabDataLoader(
    tabular_pandas.train,
    bs=128,
    shuffle=True,
    drop_last=True,)
#     num_workers=1)

val_dl = TabDataLoader(
    tabular_pandas.valid,
    bs=128,)
#     num_workers=1)

In [18]:
from fastai.data.core import DataLoaders

dls = DataLoaders(trn_dl, val_dl)

print("Sample batch:")
dls.one_batch()

Sample batch:


(tensor([[ 1,  2,  8,  ...,  2,  2,  2],
         [ 1,  3, 12,  ...,  2,  2,  2],
         [ 1,  1,  5,  ...,  2,  2,  2],
         ...,
         [ 1,  1,  1,  ...,  2,  2,  2],
         [ 2,  3, 14,  ...,  2,  2,  2],
         [ 1,  1,  3,  ...,  2,  2,  2]]),
 tensor([[-0.9668, -0.9668, -0.9665,  ..., -0.1270, -0.1117, -0.0718],
         [-0.6188, -0.6188, -0.6242,  ..., -0.1270, -0.1117, -0.0718],
         [-1.0828, -1.0828, -1.0825,  ..., -0.1270, -0.1117, -0.0718],
         ...,
         [-0.7348, -0.7348, -0.7344,  ..., -0.1270, -0.1117, -0.0718],
         [-0.3868, -0.3868, -0.3863,  ..., -0.1270, -0.1117, -0.0718],
         [-0.8508, -0.8508, -0.8504,  ..., -0.1270, -0.1117, -0.0718]]),
 tensor([[1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [3],
  

In [19]:
from fastai.metrics import F1Score, Precision, Recall, accuracy

# load the tabular_pandas data through the tabular_learner
layers = [2048, 1024, 32, 2]

# tabular learner configuration
config = tabular_config(ps=[0.2, 0.1, 0.1, 0.0], embed_p=0.1)

# import ipdb; ipdb.set_trace()
learn = tabular_learner(
    dls,
    layers=layers,
    emb_szs=emb_szs,
    # loss_func=f1_loss_func,
    config=config,
    metrics=[accuracy,
             Precision(average='macro'),
             Recall(average='macro'),
             F1Score(average='macro')])

In [20]:
learn.fit_one_cycle(n_epoch=10)

epoch,train_loss,valid_loss,accuracy,precision_score,recall_score,f1_score,time
0,2.533820,2.530330,0.007042,0.002872,0.151786,0.005046,00:09
1,2.445053,2.340379,0.049296,0.119828,0.231173,0.111262,00:09
2,1.821485,1.123364,0.692782,0.244355,0.189499,0.201871,00:09
3,0.953100,0.611989,0.802817,0.278253,0.218247,0.243321,00:09
4,0.622652,0.516502,0.845951,0.272051,0.247139,0.258956,00:10
5,0.439671,0.472592,0.855634,0.274178,0.244280,0.258122,00:09
6,0.368843,0.469667,0.854753,0.345982,0.255129,0.275320,00:10
7,0.336376,0.463960,0.852113,0.269117,0.243718,0.255687,00:09
8,0.316994,0.465880,0.852113,0.321947,0.250305,0.271537,00:09
9,0.319440,0.469549,0.854753,0.343098,0.252928,0.272802,00:09


/Users/seanmacrae/mobius/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/seanmacrae/mobius/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/seanmacrae/mobius/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/seanmacrae/mobius/.venv/lib/python3.6/site-pack

In [21]:
learn.export("tabular_learn.pkl")

In [22]:
from mobius.calibration import ModelWithTemperature

scaled_model = ModelWithTemperature(learn.model)
scaled_model.set_temperature(val_dl)
learn.model = scaled_model.model

Before temperature - NLL: 0.470, ECE: 0.044
Optimal temperature: 0.912
After temperature - NLL: 0.484, ECE: 0.047


In [23]:
# true species labels
y_true=learn.dls.valid.items["loan_status"]

# model scores and species predictions
y_scores, *_ = learn.get_preds(dl=val_dl)
preds = np.argmax(y_scores, 1).numpy()

/Users/seanmacrae/mobius/.venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
print("First 20 species labels and predictions")
list(zip(y_true, preds))[:10]

First 20 species labels and predictions


[(1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 4),
 (1, 1),
 (1, 1),
 (1, 4)]

In [25]:
(y_true == preds).sum() / len(y_true)

0.8547535211267606

In [26]:
# TODO: update roc-it with MultiClassClassification...

# clf_metrics = BinaryClassification(
#     y_true=y_true, y_scores=y_scores)

# ths = clf_metrics.recall_curve(0.95)

# clf_metrics.plot_confusion_matrix(
#     ths=0.50,
#     normalize=None,
#     save_path=f'{hyperparams["artifact_dir"]}/cm.png')

# clf_metrics.plot_pr_curve(
#     save_path=f'{hyperparams["artifact_dir"]}/pr_curve.png')

# clf_metrics.plot_roc_curve(
#     save_path=f'{hyperparams["artifact_dir"]}/roc_curve.png')

## Siamese Net

To init a new `TabularSiameseDataset` object, we only need a `tabular_pandas` object from the fast.ai library.

In [27]:
from mobius.datasets import TabularSiameseDataset

# sds = TabularSiameseDataset(tabular_pandas)
train_ds = TabularSiameseDataset(tabular_pandas.train)
valid_ds = TabularSiameseDataset(tabular_pandas.valid)

In [28]:
train_ds.__getitem__(10)

((tensor([   2,    3,   12, 3169,    2,    2,    1,    3,    1,    0,    3,    4,
           619,   42,  492,    2,    7,    1,    0,    2,    2,    1,    2,    2,
             2,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1]),
  tensor([-3.8790e-02, -3.8790e-02, -3.8131e-02,  6.0707e-02, -4.2194e-01,
          -6.6038e-01,  1.2029e+00, -3.7138e-01, -6.6686e-01, -9.7150e-02,
           9.7295e-03, -1.2477e+00, -3.7464e-01, -8.4034e-02,  2.7299e-01,
          -1.6335e+00,  2.4945e-01,  2.5015e-01, -7.4749e-01, -7.4754e-01,
          -5.9568e-01, -9.2352e-01, -5.1084e-02, -1.6687e-02, -2.3601e-02,
          -3.1684e-01, -1.2644e-01, -1.8103e-03, -1.4228e-02, -4.6833e-03,
          -7.5840e-02, -1.4422e-01, -6.3909e-01,  1.5656e-03, -6.0226e-02,
           3.6323e+00,  2.5408e+00, -1.4292e+00,  2.1500e-01,  3.0205e+00,
          -3.2988e+00, -3.5835e+00,  3.5367e+00,  3.3780e+00, -2.2053e-01,
          -1.2702e-01, -1.1174e-01, -1.4600e+00])

In [29]:
dls = DataLoaders.from_dsets(train_ds, valid_ds, bs=32, device='cpu', num_workers=0)

Siamese net encoder is the body of the Tabular net we just trained.

In [30]:
import copy

encoder = copy.copy(learn)
encoder.model.layers = learn.model.layers[:-1]
encoder_model = encoder.model

We create a new head that doubles the input shape to the last layer of the trained Tabular net, since the loss function will now compare 2 penguins. The size of the output shape is set by...???

In [31]:
from fastai.layers import LinBnDrop

head = LinBnDrop(n_in=layers[-1]*2,
    n_out=16,  # size of output space
    bn=True,
    act=None)

In [32]:
from mobius.models import TabularSiameseModel

model = TabularSiameseModel(encoder_model, head)

In [33]:
from fastai.torch_basics import params
from mobius.losses import ContrastiveLoss

def siamese_splitter(model):
    return [params(model.encoder), params(model.head)]

def contrastive_loss_func(out, targ):
    return ContrastiveLoss(margin=0.10)(out, targ.long())

In [34]:
from fastai.learner import Learner

siamese_learner = Learner(dls,
    model,
    model_dir=".",
    loss_func=contrastive_loss_func,
    splitter=siamese_splitter)

In [ ]:
# %debug
siamese_learner.freeze()
siamese_learner.fit_one_cycle(n_epoch=10)

epoch,train_loss,valid_loss,time


In [ ]:
siamese_learner.unfreeze()

lr_min, _ = siamese_learner.lr_find()
siamese_learner.fit(n_epoch=5, lr=lr_min)

In [ ]:
siamese_learner.save("snn.pkl")

In [ ]:
# TODO: train kNN on encoded penguins_train (df)

In [ ]:
from tqdm import tqdm

train_encoded = list()
for i in tqdm(range(len(dls.train_ds.tabular_pandas))):
    # get ith point from the training set
    p, _, _ = dls.train_ds.get_items(i, 0)

    # rehsape into mini-batch size 1
    p = p[0].reshape(1, -1), p[1].reshape(1, -1)

    # encode the household into output embedding space
    p_encode = siamese_learner.model.encode(p)
    train_encoded.append(p_encode)
    
ids = dls.train.get_idxs()
y_train_labels = dls.train.tabular_pandas["loan_status"]

In [ ]:
import torch
import pandas as pd

train_encoded_df = pd.DataFrame(torch.stack(train_encoded).squeeze())

In [ ]:
train_encoded_df

In [ ]:
# train_encoded_df[0].values

In [ ]:
import seaborn as sns
from sklearn.manifold import TSNE
sns.set(rc={'figure.figsize':(11.7,8.27)})

sns.scatterplot(x=train_encoded_df[0].values,
                y=train_encoded_df[1].values,
                hue=y_train_labels, 
                legend='full', 
                palette="tab10")

In [ ]:
import seaborn as sns
from sklearn.manifold import TSNE
sns.set(rc={'figure.figsize':(11.7,8.27)})

tsne = TSNE()
encoded_train_tsne = tsne.fit_transform(train_encoded_df.values)

In [ ]:
sns.scatterplot(x=encoded_train_tsne[:,0],
                y=encoded_train_tsne[:,1],
                hue=y_train_labels, 
                legend='full', 
                palette="tab10")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(train_encoded_df.values, y_train_labels.values)

We have now fit 3 models to predict penguin species - first, a multi-class Tabular learner was fit on penguins; then, a Siamese net was fit to discern likeness between pairs of penguins; lastly, a kNN supervised clustering algorithm was applied to the output of the Siamese Encoder (the manifold space) to predict species.

## Validation Set

In [ ]:
from tqdm import tqdm

valid_encoded = list()
for i in tqdm(range(len(dls.valid_ds.tabular_pandas))):
    # get ith point from the training set
    p, _, _ = dls.valid_ds.get_items(i, 0)

    # rehsape into mini-batch size 1
    p = p[0].reshape(1, -1), p[1].reshape(1, -1)

    # encode the household into output embedding space
    p_encode = siamese_learner.model.encode(p)
    valid_encoded.append(p_encode)
    
valid_ids = dls.valid.get_idxs()
y_valid_labels = dls.valid.tabular_pandas["loan_status"]

In [ ]:
valid_encoded_df = pd.DataFrame(torch.stack(valid_encoded).squeeze())

In [ ]:
encoded_valid_tsne = tsne.fit_transform(valid_encoded_df.values)

In [ ]:
sns.scatterplot(x=encoded_valid_tsne[:,0],
                y=encoded_valid_tsne[:,1],
                hue=y_valid_labels, 
                legend='full', 
                palette="tab10")

In [ ]:
res = pd.DataFrame(zip(knn.predict(valid_encoded_df), y_valid_labels), columns=["pred", "true"])

In [ ]:
(res["pred"] == res["true"]).sum() / len(res)

In [ ]:
res

In [ ]:
res[res["true"] == 3]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as multiclass_eval

prec, recall, fbeta, _ = multiclass_eval(res["true"].values, res["pred"].values, average="macro")

f"precision {prec}, recall {recall}, fbeta {fbeta}"

In [ ]:
import umap
import matplotlib.pyplot as plt

reducer = umap.UMAP()

embedding = reducer.fit_transform(valid_encoded_df.values)
embedding.shape

plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sns.color_palette()[x] for x in y_valid_labels.values])

plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the LC Loans', fontsize=24)